In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, TensorDataset
import pandas as pd

import pickle, os, sys

In [2]:

# imagebind
from imagebind import data
import encoder.custom_ib_model as cibm
from encoder.custom_ib_model import ModalityType

c:\Users\esill\miniconda3\envs\imagebind\lib\site-packages\torchaudio\backend\utils.py:62: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")
c:\Users\esill\miniconda3\envs\imagebind\lib\site-packages\torchvision\transforms\_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
c:\Users\esill\miniconda3\envs\imagebind\lib\site-packages\torchvision\transforms\_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [3]:
# mobile_net
import classifier.custom_mobile_net as cmn 

In [4]:
sys.path

['c:\\Users\\esill\\OneDrive\\Desktop\\SILK_ROAD\\Steppe-Road\\classifier',
 'c:\\Users\\esill\\OneDrive\\Desktop\\SILK_ROAD\\Steppe-Road\\encoder',
 'c:\\Users\\esill\\OneDrive\\Desktop\\SILK_ROAD\\Steppe-Road',
 'c:\\Users\\esill\\miniconda3\\envs\\imagebind\\python38.zip',
 'c:\\Users\\esill\\miniconda3\\envs\\imagebind\\DLLs',
 'c:\\Users\\esill\\miniconda3\\envs\\imagebind\\lib',
 'c:\\Users\\esill\\miniconda3\\envs\\imagebind',
 '',
 'c:\\Users\\esill\\miniconda3\\envs\\imagebind\\lib\\site-packages',
 'c:\\Users\\esill\\miniconda3\\envs\\imagebind\\lib\\site-packages\\imagebind',
 'c:\\Users\\esill\\miniconda3\\envs\\imagebind\\lib\\site-packages\\win32',
 'c:\\Users\\esill\\miniconda3\\envs\\imagebind\\lib\\site-packages\\win32\\lib',
 'c:\\Users\\esill\\miniconda3\\envs\\imagebind\\lib\\site-packages\\Pythonwin']

In [5]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [6]:
# 데이터 준비 및 모델 로드
df = pd.read_csv('./data/data.csv')

In [7]:
def get_model(device):
    if os.path.exists('./data/cib_model.pkl'):
        with open('./data/cib_model.pkl', 'rb')as f:
            cib_model = pickle.load(f)
        
        with open('./data/embedbings.pkl', 'rb')as f:
            embeddings = pickle.load(f)
        
        cib_model.eval()
        cib_model.to(device)    
        return cib_model, embeddings
    
    cib_model = cibm.cibm_pretrained(out_embed_dim=1024)
    cib_model.eval()
    cib_model.to(device)

# 데이터 로드 및 변환
    inputs = {
        ModalityType.TEXT: data.load_and_transform_text(df['portion'], device),
        ModalityType.VISION: data.load_and_transform_vision_data(df['image'], device),
    }
    with torch.no_grad():
        embeddings = cib_model(inputs)

    with open('./data/cib_model.pkl', 'wb')as f:
        pickle.dump(cib_model, f)
    
    with open('./data/embedbings.pkl', 'wb')as f:
        pickle.dump(embeddings, f)

    return cib_model, embeddings


In [8]:
cib_model, embeddings = get_model(device=device)


In [9]:
# Average the embeddings
features = (embeddings['text'] + embeddings['vision']) / 2

# Step 2: Create dataset
softness = torch.tensor(df['softness'].values, dtype=torch.float32)
roughness = torch.tensor(df['roughness'].values, dtype=torch.float32)
thickness = torch.tensor(df['thickness'].values, dtype=torch.float32)
flexibility = torch.tensor(df['flexibility'].values, dtype=torch.float32)

# Combine features and labels into a single dataset
labels = torch.stack(softness, dim=1)
dataset = TensorDataset(features, labels)


In [10]:
input_size = 1024
model = cmn.CustomMobileNet(input=1024, num_calss = 5)
model.set_transfer_learn()
model.to(device)

c:\Users\esill\miniconda3\envs\imagebind\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\esill\miniconda3\envs\imagebind\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


CustomMobileNet(
  (input_preprocessor): CustomPreprosessor(
    (fc): Sequential(
      (0): Linear(in_features=1024, out_features=1024, bias=True)
    )
    (processor): Sequential(
      (0): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
  )
  (mobilenet): MobileNetV3(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): Hardswish()
      )
      (1): InvertedResidual(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
            (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): R

In [11]:
# 데이터셋 분할
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [12]:
# 손실 함수 및 최적화 함수 정의
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [13]:
# 모델 학습 및 검증
num_epochs = 25
best_val_loss = float('inf')

In [15]:
train_loss = []
val_loss = []
if os.path.exists('best_model_1024.pth'):
    model.load_state_dict(torch.load('best_model_1024.pth'), )
for epoch in range(num_epochs):
    # train, valid loss 계산
    train_loss.append(cmn.train_model(model, train_loader, criterion, optimizer, device, num_epochs=5))
    val_loss.append(cmn.evaluate_model(model, val_loader, criterion, device))
    # valid_loss 최솟값 저장
    if val_loss[-1] < best_val_loss:
        best_val_loss = val_loss[-1]
        print(f'new {best_val_loss}', flush=True)
        torch.save(model.state_dict(), f'best_model_{input_size}.pth')
    print(f'epoch {epoch} train {train_loss[-1]} valid {val_loss[-1]}', end = '\r', flush=True)


new 2.7452762126922607
new 2.34767198562622072892972507 valid 2.7452762126922607
new 2.04481196403503423202397275 valid 2.3476719856262207
new 1.8209196329116821031651815 valid 2.044811964035034
new 1.74732518196105960267199004 valid 2.4603254795074463
new 1.71030712127685554137123303 valid 1.7473251819610596
new 1.288522958755493260582420154 valid 1.8302799463272095


In [ ]:
best_val_loss

0.48258474469184875